In [377]:
import feature_extraction as fe

import numpy as np

import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
feats, labels = fe.extract_numfeats('../datasets/GTZAN.gui')

Currently processing:  100
Currently processing:  200
Currently processing:  300
Currently processing:  400
Currently processing:  500
Currently processing:  600
Currently processing:  700
Currently processing:  800
Currently processing:  900
Currently processing:  1000


In [384]:
scaler = StandardScaler()
scaled_feats = scaler.fit_transform(feats)

In [385]:
x_trainval, x_test, y_trainval, y_test = train_test_split(scaled_feats, labels, test_size=0.2)
x_train, x_val, y_train, y_val = train_test_split(x_trainval, y_trainval, test_size=0.25)

In [386]:
train_data = [(x, y) for x, y in zip(x_train, y_train)]
val_data = [(x, y) for x, y in zip(x_val, y_val)]
test_data = [(x, y) for x, y in zip(x_test, y_test)]

print(len(train_data), "training samples")
print(len(val_data), "validation samples")
print(len(test_data), "test samples")

600 training samples
200 validation samples
200 test samples


In [414]:
seed = 1111
torch.manual_seed(seed)

# Hyper-parameters 
input_size = 25
#hidden_size = [256, 128, 64]
hidden_size = [25, 20, 15]
num_classes = 10
num_epochs = 50
batch_size = 50
learning_rate = 0.001

# Fully connected neural network with one hidden layer
class DNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(DNNClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size[0])
        self.fc2 = nn.Linear(hidden_size[0], hidden_size[1])
        self.fc3 = nn.Linear(hidden_size[1], hidden_size[2])
        self.fc4 = nn.Linear(hidden_size[2], num_classes)
    
    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = F.relu(self.fc2(out))
        out = F.relu(self.fc3(out))
        out = F.relu(self.fc4(out))
        return out # we can return here the softmax, but then we should use the nll_loss instead of the cross_entropy
    

model = DNNClassifier(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [415]:
def data_chunker(data, batch_size):
    random.shuffle(data)
    return [data[i:i + batch_size] for i in range(0, len(data), batch_size)]

In [418]:
# Train the model
total_step = len(train_data)
for epoch in range(num_epochs):
    #for i, (features, label) in enumerate(train_data):
    data_chunks = data_chunker(train_data, batch_size)
    for i, chunk in enumerate(data_chunks):
        x, y = list(zip(*chunk))
        
        # Move tensors to the configured device
        features = torch.from_numpy(np.array(x)).float().to(device)
        #label = torch.from_numpy(np.array([label])).to(device)
        lbls = torch.from_numpy(np.array(y)).to(device)

        # clear all the gradients of the optimized tensors
        optimizer.zero_grad()

        # Forward pass
        output = model(features)
        loss = criterion(output, lbls)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        if (i+1)*batch_size % 200 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, (i+1)*batch_size, total_step, loss.item()))

Epoch [1/50], Step [200/600], Loss: 1.3940
Epoch [1/50], Step [400/600], Loss: 1.3943
Epoch [1/50], Step [600/600], Loss: 1.2809
Epoch [2/50], Step [200/600], Loss: 1.3020
Epoch [2/50], Step [400/600], Loss: 1.4882
Epoch [2/50], Step [600/600], Loss: 1.7029
Epoch [3/50], Step [200/600], Loss: 1.7375
Epoch [3/50], Step [400/600], Loss: 1.5023
Epoch [3/50], Step [600/600], Loss: 1.5532
Epoch [4/50], Step [200/600], Loss: 1.6633
Epoch [4/50], Step [400/600], Loss: 1.7070
Epoch [4/50], Step [600/600], Loss: 1.3809
Epoch [5/50], Step [200/600], Loss: 1.6638
Epoch [5/50], Step [400/600], Loss: 1.7925
Epoch [5/50], Step [600/600], Loss: 1.4010
Epoch [6/50], Step [200/600], Loss: 1.6568
Epoch [6/50], Step [400/600], Loss: 1.2721
Epoch [6/50], Step [600/600], Loss: 1.4488
Epoch [7/50], Step [200/600], Loss: 1.6293
Epoch [7/50], Step [400/600], Loss: 1.5300
Epoch [7/50], Step [600/600], Loss: 1.6143
Epoch [8/50], Step [200/600], Loss: 1.4265
Epoch [8/50], Step [400/600], Loss: 1.7203
Epoch [8/50

In [419]:
with torch.no_grad():
    correct = 0
    total = 0

    features = torch.from_numpy(x_val).float().to(device)
    lbls = torch.from_numpy(y_val).to(device)
    outputs = model(features)
    _, predicted = torch.max(outputs.data, 1)
    total += lbls.size(0)
    correct += (predicted == lbls).sum().item()

    print('Accuracy: {} %'.format(100 * correct / total))

Accuracy: 45.5 %
